# Process Globe-LFMC samples and extract DEM Data
Extracts the samples for locations in the CONUS from the Globe-LFMC spread sheet and adds the normalised DEM, climate zone, and other auxiliary data. DEM data is from the GEE SRTM DEM product, extracted using the MODIS projection and scale. Sites within the same MODIS pixel are merged. The following files are created:
- `LFMC_CONUS.csv`: CONUS data extracted from the Globe-LFMC dataset
- `LFMC_sites.csv`: sites extracted from the Globe-LFMC CONUS data and augmented with normalised DEM and location data
- `LFMC_samples.csv`: Globe-LFMC CONUS sample data augmented with auxiliary variables

### Notes
1. The `Globe-LFMC-v2.xlsx` should exist in the `INPUT_DIR` directory - by default, a sub-directory of `DATA_DIR`
2. The tiff containing the Koppen climate zone data (`Beck_KG_V1_present_0p0083.tif` available from https://figshare.com/articles/dataset/Present_and_future_K_ppen-Geiger_climate_classification_maps_at_1-km_resolution/6396959/2) should also be in `INPUT_DIR`, as should either the `legend.txt` file (available from the same site) or `Climate_zones.csv`. If `Climate_zones.csv` doesn't exist, it needs to be created from `legend.txt` by uncommenting and running the first code cell under "Climate zone processing".
3. `EXTRACT_NAME` is a sub-directory of `DATA_DIR`. It will be created if it doesn't exist. All data files created by this and other data extraction notebooks will be located in sub-directories of this directory.
4. `LFMC_CONUS.csv` is created in the `INPUT_DIR` directory.
5. All other created files are CSVs and stored in the `SAMPLE_DIR` directory, by default a sub-directory of `DATA_DIR/EXTRACT_NAME`.
6. The samples data output by this code is further processed by the MODIS extraction code to remove the snow samples.

In [ ]:
import os
import numpy as np
import pandas as pd
import time
from datetime import datetime
from datetime import timedelta

import initialise
import common
from data_extract_utils import normalise_dem
from data_extract_utils import extract_koppen_data
from data_prep_utils import normalise

Define input and output files

In [ ]:
# Globe-LFMC file and sheet name
GLOBE_LFMC = os.path.join(common.SOURCE_DIR, "Globe-LFMC-v2.xlsx")
SHEET_NAME = "LFMC data"

# File Names
LFMC_RAW = os.path.join(common.SOURCE_DIR, "LFMC_CONUS.csv")                   # CSV of CONUS data extracted from the Globe-LFMC dataset
KOPPEN_TIF = os.path.join(common.SOURCE_DIR, 'Beck_KG_V1_present_0p0083.tif')  # Tiff of Koppen climate zone values
LEGEND_FILE = os.path.join(common.SOURCE_DIR, 'legend.txt')                    # Text file with Koppen climate zone legend
KOPPEN_LEGEND = os.path.join(common.SOURCE_DIR, 'Climate_zones.csv')           # CSV of Koppen climate zone legend

if not os.path.exists(common.DATASETS_DIR):
    os.makedirs(common.DATASETS_DIR)


Other constants/parameters

In [ ]:
# DEM Product, projection and resolution
DEM_PRODUCT = 'USGS/SRTMGL1_003'
DEM_PROJ = "EPSG:4326"
DEM_SCALE = 30

# Floating point precision
FLOAT_PRE = 5

Initialise Google Earth Engine

In [ ]:
import ee
ee.Initialize()

## Point-based Processing
Extracts the DEM data from GEE usong the native DEM projection and resolution. Keeps the sample site latitude and longitude, and adds the elevation/slope/aspect.
- Parameter:
 - sites: Dataframe of sample sites
- Returns: Dataframe of sites, latitude and longitude and the added elevation/slope/aspect attributes

In [ ]:
def sites_by_point(sites):
    dem_image = ee.Terrain.products(ee.Image(DEM_PRODUCT))
    points = [ee.Geometry.Point(site.Longitude, site.Latitude) for x, site in sites.iterrows()]
    dem_col = ee.ImageCollection(dem_image)
    col_list = [dem_col.getRegion(point, DEM_SCALE, DEM_PROJ) for point in points]
    dem_list = ee.List(col_list).getInfo()
    dem_data = pd.DataFrame([item[1] for item in dem_list], columns=dem_list[0][0])
    dem_data.id = sites.Site
    dem_data.rename(columns={"id": "Site"}, inplace=True)
    dem_df = sites.merge(dem_data[['Site', 'elevation', 'slope', 'aspect']])
    dem_df.columns = ['Site', 'Latitude', 'Longitude', 'Elevation', 'Slope', 'Aspect']
    return dem_df

## Pixel-based Processing
Extracts the DEM data at the requested projection and resolution. Terrain.products adds the slope and aspect. A reducer is used so terrain product info is added before resampling.
- Parameters:
 - sites: dataframe of sampling sites
 - scale/proj: the required scale/proj (e.g. MODIS scale/proj - or map scale/proj)
 - maxPixels: Reducer parameter specifying the maximum number of DEM pixels to use to compute each down-sampled pixel. Doesn't need to be exact but make sure it's large enough - 512 is good for MODIS
- Returns: Dataframe of sites with latitude and longitude set to the pixel centroid as returned by GEE and the added elevation/slope/aspect attributes

In [ ]:
def sites_by_pixel(sites, scale, proj, maxPixels):
    dem_image = ee.Terrain.products(ee.Image(DEM_PRODUCT)).reduceResolution(ee.Reducer.mean(), maxPixels=maxPixels)
    points = [ee.Geometry.Point(site.Longitude, site.Latitude) for x, site in sites.iterrows()]
    dem_col = ee.ImageCollection(dem_image)
    col_list = [dem_col.getRegion(point, scale, proj) for point in points]
    dem_list = ee.List(col_list).getInfo()
    dem_data = pd.DataFrame([item[1] for item in dem_list], columns=dem_list[0][0])
    dem_data.id = sites.Site
    dem_data.columns = ['Site', 'Longitude', 'Latitude', 'time', 'Elevation', 'Slope', 'Aspect', 'hillshade']
    dem_df = dem_data.drop(columns=["time", "hillshade"])
    return dem_df

## Main Processing
- If the LFMC_RAW file already exists, load it.
- Otherwise extract Globe LFMC data from the excel workbook sheet and save to the LFMC_RAW file.

In [ ]:
if os.path.exists(LFMC_RAW):
    LFMC_data = pd.read_csv(LFMC_RAW, index_col=0, float_precision="high", parse_dates=["Sampling date"],
                           dtype={8: str, 10: np.int32, 11: np.int16, 14: np.int16, 23: str})
else:    
    LFMC_data = pd.read_excel(GLOBE_LFMC, SHEET_NAME).dropna(how="all")
    LFMC_data = LFMC_data[(LFMC_data.Country == "USA")
                          & (LFMC_data["State/Region"] != "Alaska")
                          & (LFMC_data["Sampling date"] >= common.START_DATE)]
    LFMC_data.to_csv(LFMC_RAW)
    LFMC_data = LFMC_data.astype(dtype={'Sampling year': np.int32, 'Protocol': np.int16, 'Units': np.int16})
LFMC_data

### Site processing
Extract the unique sites from the Globe-LFMC data

In [ ]:
LFMC_data["Site"] = LFMC_data.ID.str.rsplit("_", 1, expand=True)[0]
sites = LFMC_data[["Site", "Latitude", "Longitude"]].drop_duplicates().reset_index(drop=True)
sites

Retrieve the DEM data from GEE - run either sitesByPixel (pixel mode) or sitesByPoint (point mode)

In [ ]:
dem_df = sites_by_pixel(sites, common.SCALE, common.PROJ, 512)
dem_df

Normalise the DEM data and save the sites data. Note: sites with same latitude/longitude are *not* merged yet

In [ ]:
dem_norm = normalise_dem(dem_df.set_index('Site'), input_columns=['Longitude', 'Latitude', 'Elevation', 'Slope', 'Aspect'], precision=FLOAT_PRE)
dem_norm = dem_norm.reset_index()
dem_norm

### Date processing
Create dataframe with dates and normalised day-of-year

In [ ]:
days = pd.date_range(common.START_DATE, common.END_DATE, closed="left")
doy = pd.Series(normalise(days.dayofyear, method='range', data_range=(1, 366), scaled_range=(-np.pi, np.pi)))
days_df = pd.DataFrame({"Date": days, 
                        "Day_sin": doy.transform(np.sin).round(FLOAT_PRE),
                        "Day_cos": doy.transform(np.cos).round(FLOAT_PRE)})
days_df

### Climate zone processing

#### Create the Koppen legend csv file
If the `KOPPEN_LEGEND` file doesn't exist, uncomment and run the following cell. This will create it from the `legend.txt` file that can be downloaded with the climate zones tiff.

In [ ]:
# legend = {}
# count = 0
# with open(LEGEND_FILE) as fp:
#     for ln in fp:
#         line = ln.split(':')
#         number = line[0].strip()
#         if number.isnumeric():
#             count += 1
#             key = int(line[0].strip())
#             parts = line[1].split('[')
#             colour = parts[1].strip().strip(']').split(' ')
#             code = parts[0].strip()[:3]
#             descr = parts[0].strip()[5:]
#             value = {'Number': number, 'Code': code, 'Description': descr, 'Red': colour[0], 'Green': colour[1], 'Blue': colour[2]}
#             legend[key] = value
# legend_df = pd.DataFrame.from_dict(legend, orient='index')
# legend_df.to_csv(KOPPEN_LEGEND, index=False)

#### Extract climate zones for sites
Extract the climate zone for each site and add to the sites data.

In [ ]:
cz_columns = ['Czone1', 'Czone2', 'Czone3']
extract_koppen_data(KOPPEN_TIF, KOPPEN_LEGEND, sites, loc_columns=['Longitude', 'Latitude'], cz_columns=cz_columns)
dem_norm = dem_norm.merge(sites[['Site', 'Czone1', 'Czone2', 'Czone3']], on='Site')
dem_norm.to_csv(common.LFMC_SITES, index=False)
dem_norm

### Sample processing
Create the auxiliary dataset from the samples

##### Step 1: Merge sites and sample data to add the site longitude and latitude to the samples

In [ ]:
samples = dem_norm[["Site", "Longitude", "Latitude"]].merge(
    LFMC_data[["ID", "Site", "Sampling date", "Sampling year", "Land Cover", "LFMC value"]])
samples

##### Step 2: Merge samples for same latitude/longitude/date

In [ ]:
# Generate a common site id for each site with the same latitude and longitude
merge_columns = ["Latitude", "Longitude"]
sites_temp = dem_norm[merge_columns + ["Site"]].groupby(merge_columns, as_index=False).min()
# Merge samples for same year and location
samples = samples.merge(sites_temp, on=merge_columns, suffixes=("_x", None))
groupby_cols = ["Latitude", "Longitude", "Sampling date"]
data_cols = {"ID": "min",                                    # Unique sample ID is the first ID of the merged samples
             "Sampling year": "min",                         # They should all be the same, but need to select one
             "Land Cover": lambda x: pd.Series.mode(x)[0],   # Most common land cover value
             "LFMC value": "mean",                           # mean LFMC value
             "Site": "min"}                                  # Site id from sites_temp
samples = samples[groupby_cols + list(data_cols.keys())].groupby(groupby_cols, as_index=False).\
              agg(data_cols).round({"LFMC value": FLOAT_PRE})
samples

##### Step 3: Add the normalised auxiliary variables (day-of-year, location and DEM) to the samples

In [ ]:
aux_df = samples[["ID", "Latitude", "Longitude", "Sampling date", "Sampling year", "Land Cover", "LFMC value", "Site"]
                ].merge(days_df, left_on="Sampling date", right_on = "Date").drop(columns="Date").\
                merge(dem_norm.drop(columns=["Longitude", "Latitude"]), on="Site").sort_values("ID")
aux_df = aux_df[['ID', 'Latitude', 'Longitude', 'Sampling date', 'Sampling year', 'Land Cover', 'LFMC value', 'Site',
                 'Czone1', 'Czone2', 'Czone3',
                 'Day_sin', 'Day_cos',
                 'Long_sin', 'Long_cos', 'Lat_norm', 'Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos']]
aux_df.to_csv(common.LFMC_SAMPLES, index=False)
aux_df